In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from scipy.stats import norm,multivariate_normal

import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression



In [4]:
# logging_index
# eval_index

# logging_policy 1
# eval_policy 1
# logging_xshift 1
# eval_xshift 1
# first_component 1
# logging_xy 1
# eval_xy 1
# logging_reward 1
# eval_reward 1
# logging_label 1
# eval_reward 1



In [5]:
# box 1

# xy
# label
# fc

data = pd.read_csv('optdigits.csv',header = None)

label = np.array(data[64]) - 1
xy = np.array(data.drop([64],axis = 1))
n_class = 10
n_dim = xy.shape[1]


pca_model = PCA(n_components=1).fit(xy)
fc = pca_model.transform(xy)



In [6]:
# box 2

logging_a = 0.8
logging_b = 0.3

eval_a = 1
eval_b = 0.01

def get_distribution(fc,a,b,pca_model=None):
    m = np.min(fc)
    m_bar = np.mean(fc)
    mean = m + (m_bar-m)/a
    std = np.sqrt((m_bar - m)/b)
#     pdf = norm(loc = mean,scale = std).pdf(fc)
    pdf = np.ones(len(fc))
    return pdf/sum(pdf), mean, std








In [7]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_soften(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = alpha + u*beta
            else:
                tmp[0,t] = (1-alpha - u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [26]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_adv(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 1- alpha - u*beta
            else:
                tmp[0,t] = (alpha + u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [9]:
# def generate_policy(logging_policy_interval,eval_policy_interval):

#     p = []
#     for i in range(n_class):
#         p.append(np.random.uniform(logging_policy_interval[0],logging_policy_interval[1]))
#     p_eval = []
#     for i in range(n_class):
#         p_eval.append(np.random.uniform(eval_policy_interval[0],eval_policy_interval[1]))


#     logging_policy = np.zeros((len(xy),n_class)) 
#     for i in range(len(logging_policy)):
#         for j in range(n_class):
#             if j == label[i]:
#                 logging_policy[i,j] = p[label[i]]
#             else:
#                  logging_policy[i,j] =( 1-p[label[i]])/(n_class-1)


#     eval_policy = np.zeros((len(xy),n_class)) 
#     for i in range(len(eval_policy)):

#         for j in range(n_class):
#             if j == label[i]:
#                 eval_policy[i,j] = p_eval[label[i]]
#             else:
#                  eval_policy[i,j] =( 1-p_eval[label[i]])/(n_class-1)
#     return logging_policy, eval_policy
    


In [10]:
# sample data
# just sample index and everything can be refered from index.
def take_action(policy):
    action = random.choices([t for t in range(n_class)],policy)[0]
    prob = policy[action]
    return action, prob


def sample_logging_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    logging_index = np.zeros(size,np.int)
    logging_action = np.zeros(size,np.int)
    logging_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=pdf)[0]
        logging_index[i] = t_index
        
        action,_ = take_action(logging_policy[t_index])
        logging_action[i] = action
        
        logging_reward[i] = int(action == label[t_index])
    
    return logging_index, logging_action, logging_reward




def sample_eval_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    eval_index = np.zeros(size,np.int)
    eval_action = np.zeros(size,np.int)
    eval_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=pdf_eval)[0]
        eval_index[i] = t_index
        
        action,_ = take_action(eval_policy[t_index])
        eval_action[i] = action
        
        eval_reward[i] = int(action == label[t_index])
    
    return eval_index, eval_action, eval_reward




        



In [11]:
def p_xshift(train_x,test_x):
    from sklearn.linear_model import LogisticRegression
    X = np.concatenate((train_x,test_x),axis = 0)
#     X = np.sum(X,axis = 1,keepdims = True)
    Y = np.concatenate((np.ones(len(train_x)),np.zeros(len(test_x))),axis = 0)
    clf = LogisticRegression().fit(X,Y)
    return clf

def loggingpolicy(x,y):
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression().fit(x,y)
    return clf

In [12]:
def evaluate_ips(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy):
    
    
    
    
#     GT = np.mean(eval_reward)
    ips_list = []
    snips_list = []
    ips_r_list = []
    
    ips_list_unknown = []
    snips_list_unknown = []
    ips_r_list_unknown = []
    
    
    
    for k in range(10):
        total_reward = 0
        for i in range(len(eval_index)):
            action, prob = take_action(eval_policy[eval_index[i]])
            if action == label[eval_index[i]]:
                total_reward += 1
        GT = total_reward/len(eval_index)

        
    #     known:

        p_w = my_bound(eval_policy[logging_index, logging_action]/logging_policy[logging_index, logging_action])
        x_w = pdf_eval[logging_index]/pdf[logging_index]

        ips = p_w*logging_reward
        ips_loss = (np.mean(ips) - GT)**2
        ips_list.append(ips_loss)

        snips = p_w*logging_reward/np.mean(p_w)
        snips_loss = (np.mean(snips) - GT)**2
        snips_list.append(snips_loss)

        ips_r = x_w*p_w*logging_reward
        ips_r_loss = (np.mean(ips_r) - GT)**2
        ips_r_list.append(ips_r_loss)




    #     unknown:
        clf_xshift = p_xshift(xy[logging_index],xy[eval_index])
        clf_logging = loggingpolicy(xy[logging_index],logging_action)


        logging_predicted = clf_logging.predict_proba(xy[logging_index])[np.arange(len(logging_index)),np.array(logging_action,np.int)]
        p_w = eval_policy[logging_index, logging_action]/logging_predicted

        xshift_logging = clf_xshift.predict_proba(xy[logging_index])
        x_w = xshift_logging[:,1]/xshift_logging[:,0]



        ips = p_w*logging_reward
        ips_loss_unknown = (np.mean(ips) - GT)**2
        ips_list_unknown.append(ips_loss_unknown)

        snips = p_w*logging_reward/np.mean(p_w)
        snips_loss_unknown = (np.mean(snips) - GT)**2
        snips_list_unknown.append(snips_loss_unknown)

        ips_r = x_w*p_w*logging_reward
        ips_r_loss_unknown = (np.mean(ips_r) - GT)**2
        ips_r_list_unknown.append(ips_r_loss_unknown)

    
    return np.mean(ips_list),np.mean(snips_list),np.mean(ips_r_list),\
                np.mean(ips_list_unknown),np.mean(snips_list_unknown),np.mean(ips_r_list_unknown),
    
    

    
    
    
    
    
    
    
    
    

In [13]:
from ope_v3 import *


class WEIGHT_DATA_SET(data.Dataset):
    '''
    dataset class with instance weight 
    '''
    def __init__(self, data,reward, weights):
        '''
        weights are same dimensional witi original data
        '''
        self.data = torch.tensor(data)
        self.reward = torch.tensor(reward)
        self.weights = torch.tensor(weights)

    def __getitem__(self, index):
        img = self.data[index] 
        target = self.reward[index]
        weight = self.weights[index]
        return img, target, weight

    def __len__(self):
        return len(self.data)

# r is xshift weight
# w is policy shift weight
def train_robust_regression(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                            dm=False,xshift_known = True, policy_known = True,use_xshift = True):
#     print('train models with true density ratios')
    Myy_robust_list = []
    Myx_robust_list = []
    model_robust_list = []
    
    feature = xy[logging_index]
    eval_xshift = pdf_eval[logging_index]
    logging_xshift = pdf[logging_index]
    eval_policy = eval_policy[logging_index,logging_action]
    logging_policy = logging_policy[logging_index,logging_action]
    
    
    for i in range(n_class):
        ## generate training set for action i

        idx = (logging_action == i)


        data_i = feature[idx]
        reward_i = logging_reward[idx]
        eval_xshift_i = eval_xshift[idx]
        logging_xshift_i = logging_xshift[idx]
        eval_policy_i = eval_policy[idx]
        logging_policy_i = logging_policy[idx]

        train_size = len(data_i)

        if xshift_known:
            r = logging_xshift_i/eval_xshift_i
        if policy_known:
            w = logging_policy_i/eval_policy_i
        if use_xshift:
            w = r * w
        
        if dm:
            w = torch.ones(len(data_i))
        

        weight_st = my_bound(w)
        weighted_train = WEIGHT_DATA_SET(data_i,reward_i, w)

        train_model = Net()

        validate_size = int(0.1*train_size)

        validate_loader = data.DataLoader(data.Subset(weighted_train, range(0, validate_size)),
            batch_size=32, shuffle=True)
        # 10% validation set
        train_loader = data.DataLoader(data.Subset(weighted_train, range(validate_size, train_size)),
            batch_size=32, shuffle=True, )

        train_model, Myy, Myx, _, _, _ = train_validate_test(args,args.lr_robust, args.epochs_training, "regression", 'cpu', 'False', train_model,
            train_loader, None , validate_loader, n_class , 0.000, d = n_dim,testflag = False)

        model_robust_list.append(train_model)
        Myy_robust_list.append(Myy)
        Myx_robust_list.append(Myx)
    return model_robust_list, Myy_robust_list, Myx_robust_list




In [14]:
def predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                        logging_action,\
                       xy,  pdf, pdf_eval, logging_policy, eval_policy,\
                       dm = False,xshift_known = True, policy_known = True,use_xshift = True):
    
    
    
    if xshift_known:
        r = pdf/pdf_eval
        
    if policy_known:
        w = logging_policy/eval_policy
    
    if use_xshift:
        w = r*w
    if dm:
        w = torch.tensor([1])
    
    weight_st = my_bound(w)
    
    output = model_robust_list[int(logging_action)](torch.tensor(xy).unsqueeze(0))

    Myy = Myy_robust_list[int(logging_action)]
    Myx = Myx_robust_list[int(logging_action)]
    meanY, varY = ru.predict_regression(torch.tensor(weight_st), Myy, Myx, output, mean0, var0)
    return meanY.detach().numpy()
        
        
        

In [15]:
def evaluate_robust(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm = False,use_xshift = True):
    
    model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    for t in range(10):
        regression_list = []
        triple_r_list = []
        triple_list = []
        total_reward = 0
        for i in range(len(eval_index)):
            action, prob = take_action(eval_policy[eval_index[i]])
            if action == label[eval_index[i]]:
                total_reward += 1

            tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        action,xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],action], prob,\
                                        dm = dm,use_xshift = use_xshift)
            regression_list.append(tmp_eval)

            #     triple
            
            rw = my_bound(pdf_eval[logging_index[i]]/pdf[logging_index[i]] \
                          * eval_policy[logging_index[i],logging_action[i]]/logging_policy[logging_index[i],logging_action[i]])
            w = my_bound(eval_policy[logging_index[i],logging_action[i]]/logging_policy[logging_index[i],logging_action[i]])

            tmp = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        logging_action[i],xy[logging_index[i]], pdf[logging_index[i]], pdf_eval[logging_index[i]], \
                                        logging_policy[logging_index[i],logging_action[i]], eval_policy[logging_index[i],logging_action[i]],\
                                        dm = dm,use_xshift = use_xshift)
            
            triple_r_list.append(rw*(logging_reward[i] - tmp) + tmp_eval)

            triple_list.append(w*(logging_reward[i] - tmp) + tmp_eval)


        triple_r = np.mean(triple_r_list)
        triple = np.mean(triple_list)
        robust = np.mean(regression_list)
        GT = total_reward/len(triple_r_list)
        reg_loss.append((robust - GT)**2)
        triple_loss.append((triple - GT)**2)
        triple_r_loss.append((triple_r - GT)**2)
    
    return np.mean(reg_loss),np.mean(triple_loss),np.mean(triple_r_loss)



In [17]:
model_list = [
    'known ips',
    'known snips',
#     'known ips-r',
#     'unknown ips',
#     'unknown snips',
#     'unknown ips-r',
    'known dm',
    'known dmdr',
#     'known robust without xshift',
#     'known triple without xshift',
#     'known triple-r without xshift',
    'known robust',
    'known triple',
#     'unknown dm',
#     'unknown dmdr',
#     'unknown dmdr-r',
#     'unknown robust without xshift',
#     'unknown triple without xshift',
#     'unknown triple-r without xshift',
#     'unknown robust',
#     'unknown triple',
#     'unknown triple-r'
]
len(model_list)

6

In [28]:
# ips


# logging_a_list = [0.5,0.8,1.2]
# logging_b_list = [0.1,0.2,0.4]
size_list = [3000,4000,5000,6000]
logging_policy_list = [[0.9,0.2],[0.7,0.2],[0.5,0.2]]
# logging_policy_list = [[0.3,0.2],[0.5,0.2],[0.7,0.2]]

cc = 0    
frame = {}
frame['model'] = model_list
for size in size_list:
    for logging_interval in logging_policy_list:
        cc += 1
        print(cc)
        logging_policy, eval_policy = generate_policy_soften(logging_interval[0],logging_interval[1])

        ips = []
        snips = []
        ips_r = []

        ips_unknown = []
        snips_unknown = []
        ips_r_unknown = []

        robust = []
        triple_noxshift = []
        triple = []

        dm = []
        dmdr_noxshift = []
        dmdr = []

        robust_without_xshift = []
        triple_without_xshift = []
        triple_r_without_xshift = []

        pdf, mean_logging, std_logging = get_distribution(fc,logging_a, logging_b)
        pdf_eval, mean_eval, std_eval = get_distribution(fc,eval_a, eval_b)
        for _ in range(3):

            logging_index, logging_action, logging_reward = \
                    sample_logging_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy)
            eval_index, eval_action, eval_reward = \
                    sample_eval_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy)



            res = evaluate_ips(logging_index, logging_action, logging_reward,\
                             eval_index, eval_action, eval_reward,\
                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy)

            ips.append(res[0])
            snips.append(res[1])


            ips_unknown.append(res[3])
            snips_unknown.append(res[4])



            res = evaluate_robust(logging_index, logging_action, logging_reward,\
                                                        eval_index, eval_action, eval_reward,\
                                                        xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                                        dm = False,use_xshift = False)
            robust.append(res[0])
            triple_noxshift.append(res[1])



            res = evaluate_robust(logging_index, logging_action, logging_reward,\
                                eval_index, eval_action, eval_reward,\
                                xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                dm = True,use_xshift = False)
            dm.append(res[0])
            dmdr_noxshift.append(res[1])


        policy_name = str(size) + str(logging_interval)

        result_list = []

        result_list.append(np.sqrt(np.mean(ips)))
        result_list.append(np.sqrt(np.mean(snips)))

        result_list.append(np.sqrt(np.mean(dm)))
        result_list.append(np.sqrt(np.mean(dmdr_noxshift)))

        result_list.append(np.sqrt(np.mean(robust)))
        result_list.append(np.sqrt(np.mean(triple_noxshift)))
        frame[policy_name] = result_list
frame = pd.DataFrame(frame).round(5)
frame.to_csv('result2/newadv_noxshift.csv',index = False)


1
2
3
4
5
6
7
8
9
10
11
12
